# Activity — Optimization with SymForce

This activity has the following goals:
* Learn how to set up and solve a least-squares problem with [SymForce](https://github.com/symforce-org/symforce).
* Learn how to describe poses and transformations with [SymForce](https://github.com/symforce-org/symforce).

SymForce ([github](https://github.com/symforce-org/symforce), [docs](https://symforce.org/)) is a library for solving nonlinear least-squares problems that look like this:

$$\underset{x \in \mathcal{X}}{\text{minimize}} \quad \frac{1}{2}\sum_{i=1}^{n} \| r_i(x) \|^2. $$

Each function $r_1, \dotsc, r_n$ is called a **residual** and produces a vector of errors that we want to make small. Although we write each $r_i$ (generically) as depending on all of $x$, it will often depend (in problems we care about) only on a small subset of $x$, i.e., only on some of the variables over which we are optimizing. The game of all solvers is to take advantage of the resulting "sparsity" to find solutions much more quickly than they otherwise could be found.

I want to be clear up front that I have no special love for SymForce (or any other solver). I suggest we use it because, unlike other libraries of this sort, it has a Python interface that is reasonably easy to use and is reasonably well documented. It also allows you, if you like, to autogenerate C++ code that is competitive with other state-of-the-art solvers (e.g., [ceres](http://ceres-solver.org), [g2o](https://github.com/RainerKuemmerle/g2o), and [GTSAM](https://gtsam.org)).

I will happily abandon SymForce if something better comes along. So, I try to stay disciplined about using it *only* for optimization and not for anything else.

## Set up notebook

Do all imports.

In [ ]:
# For numerical methods
import numpy as np

# For image processing and visualization of results
import matplotlib.pyplot as plt

# For optimization with symforce
import symforce
symforce.set_epsilon_to_symbol()
import symforce.symbolic as sf
from symforce.values import Values
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
import sym

## Example: least-squares linear regression

Suppose we want to find the parameters $x_0$ and $x_1$ for which the linear model

$$b = x_0 + x_1 a$$

best fits data of the form

$$(a_0, b_0), \dotsc, (a_{n-1}, b_{n-1})$$

where $a_i, b_i \in \R$ for all $i\in\{0, \dotsc, n-1\}$. If we define "fit" in the least-squares sense, then it is equivalent to find the parameters $x_0$ and $x_1$ that solve the following least-squares problem:

$$\underset{x_0\in\R, \; x_1\in\R}{\text{minimize}} \quad \frac{1}{2}\sum_{i=1}^{n-1} \| b_i - (x_0 + x_1 a_i) \|^2.$$

If we define

$$ r(x_0, x_1, a, b) = b - (x_0 + x_1 a) $$

then can write this problem in standard form as

$$\underset{x\in\R^2}{\text{minimize}} \quad \frac{1}{2}\sum_{i=0}^{n-1} \| r_i(x) \|^2$$

where

$$ r_i(x) = r(x_0, x_1, a_i, b_i) \quad\text{for all}\quad i\in\{0, \dotsc, n-1\}. $$

This problem is a useful example because it has an analytical solution. First, you can check that

$$\frac{1}{2}\sum_{i=0}^{n-1} \| r_i(x) \|^2 = \frac{1}{2} \left\| y - A x  \right\|^2$$

where

$$y = \begin{bmatrix} b_0 \\ \vdots \\ b_{n-1} \end{bmatrix} \qquad\qquad A = \begin{bmatrix} 1 & a_0 \\ \vdots & \vdots \\ 1 & a_{n-1} \end{bmatrix} \qquad\qquad x = \begin{bmatrix} x_0 \\ x_1 \end{bmatrix}.$$

A necessary condition for optimality is

$$\begin{align*} 0 &= \left( \dfrac{\partial \left( \frac{1}{2} \left\| y - A x \right\|^2 \right) }{\partial x} \right)^T \\ &= \left( \left(y - A x \right)^T A \right)^T \\ &= A^T \left(y - A x \right) \\ &= A^Ty - A^TA x \end{align*}$$

and so the solution, should one exist, is given by

$$ x = \left( A^T A \right)^{-1} A^T y. $$

You'll notice the famous pseudo-inverse

$$\left( A^T A \right)^{-1} A^T$$

appearing in this expression. In practice, you would never want to compute this pseudo-inverse explicitly (the usual thing to do instead is use the QR decomposition), but it will be fine for our example.

#### Generate a dataset

Create a random number generator with a particular seed so that our results are reproducible.

In [ ]:
rng = np.random.default_rng(0)

Choose $x$.

In [ ]:
x_0 = 0.2
x_1 = 0.7

Sample $a$ and $b$.

In [ ]:
# Number of points
n = 10

# Sample a_i from a uniform distribution on the unit interval
a = rng.uniform(size=n)

# Sample b_i from a normal distribution with mean x_0 + x_1 * a_i and standard deviation sigma
sigma = 0.02
b = x_0 + x_1 * a + sigma * rng.standard_normal(size=n)

#### Find the analytical solution

Get the solution.

In [ ]:
# Find y
y = b.copy()

# Find A
A = np.column_stack([np.ones_like(a), a])

# Estimate x
x_analytical = np.linalg.inv(A.T @ A) @ A.T @ y

Visualize the solution.

In [ ]:
fig, ax = plt.subplots(1, 1)
at = np.linspace(0., 1.)
bt = x_analytical[0] + x_analytical[1] * at
ax.plot(at, bt, '-')
ax.plot(a, b, '.', markersize=9)
ax.set_aspect('equal')
ax.set_xlim(0., 1.)
ax.set_ylim(0., 1.)
plt.show()


#### Find the numerical solution

Define the residual function $r(x_0, x_1, a, b)$.

In [ ]:
def residual(
    x_0: sf.Scalar,
    x_1: sf.Scalar,
    a: sf.Scalar,
    b: sf.Scalar,
    epsilon: sf.Scalar,
) -> sf.V1:
    return sf.V1(b - (x_0 + x_1 * a))

There are two important things to note about this residual function.

First, it uses [typing](https://docs.python.org/3/library/typing.html). Every input is listed with its type (e.g., `x_1` is an `sf.Scalar`, which is a number). The output is also given a type (`sf.V1`, which is a vector of length one). These types are prefixed by `sf` because they are defined by SymForce.

Second, it is symbolic. In particular, we can evaluate this function with symbolic arguments:

In [ ]:
# Find symbolic residual
r = residual(
    sf.Symbol('x_0'),
    sf.Symbol('x_1'),
    sf.Symbol('a_i'),
    sf.Symbol('b_i'),
    sf.epsilon,
)

# Show symbolic residual
print(r)

Just like with [SymPy](https://www.sympy.org/), on which SymForce is based, we can create a numeric version of this symbolic function using `lambdify`:

In [ ]:
residual_num = symforce.util.lambdify(residual)

Suppose, for example, we wanted to find the error

$$r_0(x) = r(x_0, x_1, a_0, b_0)$$

that is associated with our analytical estimate of $x_0$ and $x_1$:

In [ ]:
# Find numeric residual
r = residual_num(
    x_analytical[0],
    x_analytical[1],
    a[0],
    b[0],
    sym.epsilon,
)

# Show numeric residual
print(r)

# Check that numeric residual is what we expect it to be
assert(np.isclose(r.item(), b[0] - (x_analytical[0] + x_analytical[1] * a[0])))

You'll have noticed that one argument of the residual function is called `epsilon`. This argument says how close a number needs to be to zero in order to be considered simply zero. It's a tolerance, in other words. Including it allows us to handle singularities — i.e., situations in which a function needs to be computed in a different way when one or more inputs take on certain values. Even if we don't use `epsilon` (as in this example), we always need to include it as the last argument to any residual function defined in SymForce.

The symbolic version of `epsilon` is `sf.epsilon`. The numeric version of `epsilon` is `sym.epsilon`. Yes, this naming convention (prefixing the numeric version with `sym`) is annoying. Let's see what these two things are:

In [ ]:
print(f'sf.epsilon = {sf.epsilon} (of type {type(sf.epsilon)})')
print(f'sym.epsilon = {sym.epsilon} (of type {type(sym.epsilon)})')

Having defined our residual function, the next step is to define a set of "initial values" that holds **all** problem data — both the unknowns ($x_0$ and $x_1$) and the knowns ($a_0, \dotsc, a_{n-1}$ and $b_0, \dotsc, b_{n-1}$, as well as `epsilon`).

In [ ]:
# Define initial values with a placeholder (an empty list) to hold pairs a_i, b_i
initial_values = Values(
    data=[],
    x_0=1.,                  # <-- initial guess
    x_1=1.,                  # <-- initial guess
    epsilon=sym.epsilon,
)

# Add each pair a_i, b_i to the list of data
for i in range(n):
    initial_values['data'].append(Values(
        a=a[i],             # <-- a_i
        b=b[i],             # <-- b_i
    ))

You can access values the same way you would access elements of a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries). Here are some examples:

In [ ]:
# Get x_0 and x_1
print('x_0 and x_1:')
print(initial_values['x_0'])
print(initial_values['x_1'])
print('')

# Get a_3 and b_3
print('a_3 and b_3:')
print(initial_values['data'][3]['a'])
print(initial_values['data'][3]['b'])

You can also access a nested value "by name" with a single string as follows (see the [documentation on Values](https://symforce.org/tutorials/values_tutorial.html) for more information). Here are two examples:

In [ ]:
print('a_3 and b_3:')
print(initial_values['data[3].a'])
print(initial_values['data[3].b'])


Next, we need to define the full cost function that is being minimized. In other words, for each element $i$ of the sum
$$\sum_{i=0}^{n-1} \left\| r_i(x) \right\|^2$$
we need to say (1) what $r_i$ is and (2) what data $r_i$ depends on. These two things together are called a **factor**.

In [ ]:
factors = []                    # <-- factors are stored in a list
for i in range(n):              # <-- there are n of them (i.e., n elements of the sum)
    factors.append(Factor(      # <-- each factor is appended to the list as it is created
        residual=residual,      # <-- what r_i is (i.e., the name of the residual function to
                                #     which it corresponds - always the same in this example)
        keys=[                  # <-- what r_i depends on (i.e., a list that has the names of
                                #     the variables at which r_i is evaluated - these must be
                                #     listed in the order of r_i's arguments, and must be the
                                #     names we defined in "initial_values")
            'x_0',              # <-- the first argument of r_i is x_0
            'x_1',              # <-- the second argument of r_i is x_1
            f'data[{i}].a',     # <-- the third argument of r_i is a_i, the "name" of which is data[i].a
            f'data[{i}].b',     # <-- the fourth argument of r_i is b_i, the "name" of which is data[i].b
            'epsilon',          # <-- the last argument of r_i is always epsilon
        ]
    ))

Next, we need to create an **optimizer**:

In [ ]:
optimizer = Optimizer(
    factors=factors,                    # <-- list of factors
    optimized_keys=['x_0', 'x_1'],      # <-- list of decision variables (names from "initial_values")
    debug_stats=True,                   # <-- whether or not to compute extra information about each iteration
)

Finally, we ask the optimizer to solve the least-squares problem that we defined.

In [ ]:
result = optimizer.optimize(initial_values)
assert(result.status == Optimizer.Status.SUCCESS)

Let's see what `result` contains.

In [ ]:
result?

You see that `result` has an attribute called `optimized_values` that has exactly the same structures as `initial_values` but with values *after* optimization. In particular, assuming the optimizer was successful, we can find the estimates of $x_1$ and $x_2$ as follows:

In [ ]:
# Get the (numerical) solution
x_numerical = np.array([
    result.optimized_values['x_0'],
    result.optimized_values['x_1'],
])

# Check that the numerical solution is the same as the analytical solution
assert(np.allclose(x_numerical, x_analytical))

Visualize the solution.

In [ ]:
fig, ax = plt.subplots(1, 1)
at = np.linspace(0., 1.)
bt_a = x_analytical[0] + x_analytical[1] * at
bt_n = x_numerical[0] + x_numerical[1] * at
ax.plot(at, bt_a, '-', color='C0', label='analytical', linewidth=1)
ax.plot(at, bt_n, '--', color='C2', label='numerical', linewidth=3)
ax.plot(a, b, '.', markersize=9, color='C1')
ax.set_aspect('equal')
ax.set_xlim(0., 1.)
ax.set_ylim(0., 1.)
ax.legend()
plt.show()

In summary, here are all the steps to find a numerical solution to the least-squares linear regression problem.

In [ ]:
# Define residual function
def residual(
    x_0: sf.Scalar,
    x_1: sf.Scalar,
    a: sf.Scalar,
    b: sf.Scalar,
    epsilon: sf.Scalar,
) -> sf.V1:
    return sf.V1(b - (x_0 + x_1 * a))

# Define initial values
initial_values = Values(
    data=[],
    x_0=1.,
    x_1=1.,
    epsilon=sym.epsilon,
)
for i in range(n):
    initial_values['data'].append(Values(
        a=a[i],
        b=b[i],
    ))

# Define factors
factors = []
for i in range(n):
    factors.append(Factor(
        residual=residual,
        keys=[
            'x_0',
            'x_1',
            f'data[{i}].a',
            f'data[{i}].b',
            'epsilon',
        ]
    ))

# Create optimizer
optimizer = Optimizer(
    factors=factors,
    optimized_keys=['x_0', 'x_1'],
    debug_stats=True,
)

# Run optimizer
result = optimizer.optimize(initial_values)
assert(result.status == Optimizer.Status.SUCCESS)

# Get solution
x_numerical = np.array([
    result.optimized_values['x_0'],
    result.optimized_values['x_1'],
])

## Example: second-order polynomial regression

Suppose we want to find the parameters $x_0, x_1, x_2 \in \R$ for which the second-order polynomial

$$b = x_0 + x_1 a + x_2 a^2$$

best fits (in the least-squares sense) data of the form

$$(a_0, b_0), \dotsc, (a_{n-1}, b_{n-1})$$

where $a_i, b_i \in \R$ for all $i\in\{0, \dotsc, n-1\}$. If we define

$$ r(x_0, x_1, x_2, a, b) = b - (x_0 + x_1 a + x_2 a^2) $$

then it is equivalent to solve the least-squares problem

$$\underset{x\in\R^2}{\text{minimize}} \quad \frac{1}{2}\sum_{i=0}^{n-1} \| r_i(x) \|^2$$

where

$$ r_i(x) = r(x_0, x_1, x_2, a_i, b_i) \quad\text{for all}\quad i\in\{0, \dotsc, n-1\}. $$

The analytical solution to this problem is

$$ x = \left( A^T A \right)^{-1} A^T y $$

where

$$y = \begin{bmatrix} b_0 \\ \vdots \\ b_{n-1} \end{bmatrix} \qquad\qquad A = \begin{bmatrix} 1 & a_0 & a_0^2 \\ \vdots & \vdots \\ 1 & a_{n-1} & a_{n-1}^2 \end{bmatrix} \qquad\qquad x = \begin{bmatrix} x_0 \\ x_1 \\ x_2 \end{bmatrix}.$$

Generate a dataset.

In [ ]:
# Create a random number generator with a particular seed so that results are reproducible
rng = np.random.default_rng(1)

# Choose x
(x_0, x_1, x_2) = (0.3, -1., 2.0)

# Sample a, b
n = 10
sigma = 0.02
a = rng.uniform(size=n)
b = x_0 + x_1 * a + x_2 * a**2 + sigma * rng.standard_normal(size=n)

Find the analytical solution.

In [ ]:
# Find y
y = b.copy()

# Find A
A = np.column_stack([np.ones_like(a), a, a**2])

# Estimate x
x_analytical = np.linalg.inv(A.T @ A) @ A.T @ y

Find the numerical solution.

**FIXME - modify the following code (copy-pasted from the linear regression example) so it solves the second-order polynomial regression problem.**

In [ ]:
#
# FIXME
#

# Define residual function
def residual(
    x_0: sf.Scalar,
    x_1: sf.Scalar,
    a: sf.Scalar,
    b: sf.Scalar,
    epsilon: sf.Scalar,
) -> sf.V1:
    return sf.V1(b - (x_0 + x_1 * a))

# Define initial values
initial_values = Values(
    data=[],
    x_0=1.,
    x_1=1.,
    epsilon=sym.epsilon,
)
for i in range(n):
    initial_values['data'].append(Values(
        a=a[i],
        b=b[i],
    ))

# Define factors
factors = []
for i in range(n):
    factors.append(Factor(
        residual=residual,
        keys=[
            'x_0',
            'x_1',
            f'data[{i}].a',
            f'data[{i}].b',
            'epsilon',
        ]
    ))

# Create optimizer
optimizer = Optimizer(
    factors=factors,                    # <-- list of factors
    optimized_keys=['x_0', 'x_1'],        # <-- list of decision variables (names from "initial_values")
    debug_stats=True,                   # <-- whether or not to show information about each iteration
)

# Run optimizer
result = optimizer.optimize(initial_values)
assert(result.status == Optimizer.Status.SUCCESS)

# Get solution
x_numerical = np.array([
    result.optimized_values['x_0'],
    result.optimized_values['x_1'],
])

Confirm that the analytical and numerical solutions are the same.

In [ ]:
assert(np.allclose(x_analytical, x_numerical))

Visualize the numerical solution.

In [ ]:
fig, ax = plt.subplots(1, 1)
at = np.linspace(0., 1.)
bt = x_numerical[0] + x_numerical[1] * at + x_numerical[2] * at**2
ax.plot(at, bt, '-')
ax.plot(a, b, '.', markersize=9)
ax.set_aspect('equal')
ax.set_xlim(0., 1.)
ax.set_ylim(0., 1.)
plt.show()

## Example: point cloud alignment

Here is how we describe the pose $T_B^A$ of frame $B$ in the coordinates of frame $A$ using NumPy:

In [ ]:
# Orientation of B in A
R_inA_ofB = np.array([
    [np.cos(np.pi / 6), -np.sin(np.pi / 6), 0.],
    [np.sin(np.pi / 6), np.cos(np.pi / 6), 0.],
    [0., 0., 1.],
])

# Position of B in A
p_inA_ofB = np.array([0.5, -0.2, 0.1])

# Pose of B in A
T_inA_ofB = np.row_stack([
    np.column_stack([R_inA_ofB, p_inA_ofB]),
    np.concatenate([np.zeros(3), np.ones(1)]),
])

Here is how we describe the same pose (numerically) using SymForce:

In [ ]:
T_inA_ofB_sym = sym.Pose3(
    R=sym.Rot3.from_rotation_matrix(R_inA_ofB),
    t=p_inA_ofB,
)

Let's look at these two things.

In [ ]:
print('NumPy:')
print(T_inA_ofB)

print('')

print('SymForce:')
print(T_inA_ofB_sym)

Clearly, SymForce does not represent poses with homogeneous transformation matrices. Instead, it represents them with seven numbers — the first four are the elements of a quaternion that describes orientation, and the last three are the coordinates of position. You can recover the homogeneous transformation matrix from a `sym.Pose3` object like this (note the misspelling of "homogeneous" as "homogenous"):

In [ ]:
print(T_inA_ofB_sym.to_homogenous_matrix())
assert(np.allclose(T_inA_ofB_sym.to_homogenous_matrix(), T_inA_ofB))

Here is how we would find the inverse transformation matrix in NumPy:

In [ ]:
# The right way, given R and p
T_inB_ofA = np.row_stack([
    np.column_stack([R_inA_ofB.T, -R_inA_ofB.T @ p_inA_ofB]),
    np.concatenate([np.zeros(3), np.ones(1)]),
])

# The right way, given only T
T_inB_ofA_secondmethod = np.row_stack([
    np.column_stack([T_inA_ofB[0:3, 0:3].T, -T_inA_ofB[0:3, 0:3].T @ T_inA_ofB[0:3, 3]]),
    np.concatenate([np.zeros(3), np.ones(1)]),
])

# The wrong way (inefficient and less accurate), given only T
T_inB_ofA_thirdmethod = np.linalg.inv(T_inA_ofB)

assert(np.allclose(T_inB_ofA, T_inB_ofA_secondmethod))
assert(np.allclose(T_inB_ofA, T_inB_ofA_thirdmethod))

Here is how we would find the inverse transformation matrix in SymPy:

In [ ]:
T_inB_ofA_sym = T_inA_ofB_sym.inverse()
assert(np.allclose(T_inB_ofA_sym.to_homogenous_matrix(), T_inB_ofA))

Here is how we would do coordinate transformation (of **points**) in NumPy:

In [ ]:
p_inA = R_inA_ofB @ p_inB + p_inA_ofB
print(p_inA)

We could also do it in homogeneous coordinates like this:

In [ ]:
# Create a point
p_inB = np.array([1., 2., 3.])

# Express the point in homogeneous coordinates
p_inB_homog = np.concatenate([p_inB, np.ones(1)])

# Perform the transformation
p_inA_homog = T_inA_ofB @ p_inB_homog

# Extract the transformed point from homogeneous coordinates
p_inA = p_inA_homog[0:3]

# Show the transformed point
print(p_inA)

Here is how we would do coordinate transformation (of **points**) in SymForce — note the use of `*` instead of `@`, and note that conversion to and from homogeneous coordinates is done behind the scenes, if at all:

In [ ]:
p_inA_sym = T_inA_ofB_sym * p_inB

print(p_inA_sym)
assert(np.allclose(p_inA_sym, p_inA))

All of these operations can be performed symbolically (with `sf.` instead of `sym.`) as well. Everything is the same, *except* that we need to wrap positions in `sf.V3` instead of in a NumPy array. Here is an example (a silly one, because we are using numbers instead of symbols):

In [ ]:
# Create transformation matrix
T_inA_ofB_sf = sf.Pose3(
    R=sf.Rot3.from_rotation_matrix(R_inA_ofB),
    t=sf.V3(p_inA_ofB),
)

# Show transformation matrix
print('T_inA_ofB_sf')
print(T_inA_ofB_sf)

print('')

# Perform coordinate transformation
p_inB_sf = sf.V3(p_inB)
p_inA_sf = T_inA_ofB_sf * p_inB_sf

# Show transformed coordinates
print('p_inA_sf')
print(p_inA_sf)

Suppose we are given a description of the same $n$ points in two different frames:

$$p^{A}_0, \dotsc, p^{A}_{n-1} \qquad\text{and}\qquad p^{B}_0, \dotsc, p^{B}_{n-1}.$$

As a challenge, see if you can use SymForce to formulate and solve the following nonlinear least-squares problem:

$$\underset{T_B^A\in SE(3)}{\text{minimize}} \quad \frac{1}{2}\sum_{i=0}^{n-1} \| p^A_i - T_B^A p^B_i \|^2$$

In other words, see if you can solve for the (assumed unknown) transformation matrix $T_B^A$ that describes how the two coordinate representations of these $n$ points are related.

Create dataset.

In [ ]:
# Random number generator with particular seed
rng = np.random.default_rng(2)

# Number of points
n = 10

# How much noise
sigma = 0.02

# Sample points one by one
p_inB = []
p_inA = []
for i in range(10):
    p_inB.append(rng.uniform(size=3))
    p_inA.append(R_inA_ofB @ p_inB[-1] + p_inA_ofB + sigma * rng.standard_normal(size=3))
p_inB = np.array(p_inB)
p_inA = np.array(p_inA)

**FIXME - Formulate and solve problem with SymForce.**